In [ ]:
# try:
#     from darts import TimeSeries
#     import ipdb
# except:
# #     !pip install -q darts
#     %pip install -q git+https://github.com/thucpc/darts_thuc.git
#     %pip install -q ipdb
#     %pip install -q lightning
# #     !pip install -q optuna-integration

In [1]:
import ipdb
import glob
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics


from darts import TimeSeries, concatenate
from darts.utils.callbacks import TFMProgressBar

# from darts.models.components.glu_variants import SwiGLU
from darts.models import TiDEModel,TransformerModel,TFTModel
from darts.models.forecasting.transformer_invert import iTransformerModel
from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler,StaticCovariatesTransformer
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import logging

logging.disable(logging.CRITICAL)

import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

# for reproducibility
torch.manual_seed(1)
np.random.seed(1)


C:\Users\Delll\AppData\Roaming\Python\Python312\site-packages\statsforecast\core.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import torch
import torch.nn as nn

class SwiGLU(nn.Module):
    def __init__(self, input_dim=None, d_model=None, beta=1.0):
        super().__init__()
        self.input_dim = input_dim
        if self.input_dim is not None:
            self.linear1 = nn.Linear(self.input_dim, self.input_dim)
            self.linear2 = nn.Linear(self.input_dim, self.input_dim)
        else:
            self.linear1 = None
            self.linear2 = None
        self.beta = nn.Parameter(torch.tensor(beta))  # Biến beta thành tham số có thể học

    def forward(self, x):
        if self.linear1 is None or self.linear2 is None:
            self.linear1 = nn.Linear(x.size(-1), x.size(-1))
            self.linear2 = nn.Linear(x.size(-1), x.size(-1))
        return self.swish(self.linear1(x)) * self.linear2(x)

    def swish(self, x):
        return x * torch.sigmoid(self.beta * x)

class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))
    
class CustomLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input_value, target_value):
        error = target_value - input_value
        A=torch.sum((target_value**2)*(error**2))
        B=torch.sum(target_value**2)
        loss = A/B
        return loss

class CustomLoss1(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input_value, target_value):
        error = target_value - input_value
        loss = torch.sum(target_value*(error**2))/torch.sum(target_value)
        return loss

class CustomLoss2(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        loss=torch.sum(y_prime_t**2*torch.log(torch.cosh(ey_t**2 + 1e-12)))
        return loss/torch.sum(y_prime_t**2)


In [ ]:
# def _mean_squared_error_compute(sum_squared_error: Tensor, num_obs: Union[int, Tensor], squared: bool = True) -> Tensor:
#     """Compute Mean Squared Error.

#     Args:
#         sum_squared_error: Sum of square of errors over all observations
#         num_obs: Number of predictions or observations
#         squared: Returns RMSE value if set to False.

#     Example:
#         >>> preds = torch.tensor([0., 1, 2, 3])
#         >>> target = torch.tensor([0., 1, 2, 2])
#         >>> sum_squared_error, num_obs = _mean_squared_error_update(preds, target, num_outputs=1)
#         >>> _mean_squared_error_compute(sum_squared_error, num_obs)
#         tensor(0.2500)

#     """
#     return sum_squared_error / num_obs if squared else torch.sqrt(sum_squared_error / num_obs)

# class MeanSquaredError(Metric):
#     r"""Compute `mean squared error`_ (MSE).

#     .. math:: \text{MSE} = \frac{1}{N}\sum_i^N(y_i - \hat{y_i})^2

#     Where :math:`y` is a tensor of target values, and :math:`\hat{y}` is a tensor of predictions.

#     As input to ``forward`` and ``update`` the metric accepts the following input:

#     - ``preds`` (:class:`~torch.Tensor`): Predictions from model
#     - ``target`` (:class:`~torch.Tensor`): Ground truth values

#     As output of ``forward`` and ``compute`` the metric returns the following output:

#     - ``mean_squared_error`` (:class:`~torch.Tensor`): A tensor with the mean squared error

#     Args:
#         squared: If True returns MSE value, if False returns RMSE value.
#         num_outputs: Number of outputs in multioutput setting
#         kwargs: Additional keyword arguments, see :ref:`Metric kwargs` for more info.

#     Example::
#         Single output mse computation:

#         >>> from torch import tensor
#         >>> from torchmetrics.regression import MeanSquaredError
#         >>> target = tensor([2.5, 5.0, 4.0, 8.0])
#         >>> preds = tensor([3.0, 5.0, 2.5, 7.0])
#         >>> mean_squared_error = MeanSquaredError()
#         >>> mean_squared_error(preds, target)
#         tensor(0.8750)

#     Example::
#         Multioutput mse computation:

#         >>> from torch import tensor
#         >>> from torchmetrics.regression import MeanSquaredError
#         >>> target = tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
#         >>> preds = tensor([[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]])
#         >>> mean_squared_error = MeanSquaredError(num_outputs=3)
#         >>> mean_squared_error(preds, target)
#         tensor([1., 4., 9.])

#     """

#     is_differentiable = True
#     higher_is_better = False
#     full_state_update = False
#     plot_lower_bound: float = 0.0

#     sum_squared_error: Tensor
#     total: Tensor

#     def __init__(
#         self,
#         squared: bool = True,
#         num_outputs: int = 1,
#         **kwargs: Any,
#     ) -> None:
#         super().__init__(**kwargs)

#         if not isinstance(squared, bool):
#             raise ValueError(f"Expected argument `squared` to be a boolean but got {squared}")
#         self.squared = squared

#         if not (isinstance(num_outputs, int) and num_outputs > 0):
#             raise ValueError(f"Expected num_outputs to be a positive integer but got {num_outputs}")
#         self.num_outputs = num_outputs

#         self.add_state("sum_squared_error", default=torch.zeros(num_outputs), dist_reduce_fx="sum")
#         self.add_state("total", default=tensor(0), dist_reduce_fx="sum")

#     def update(self, preds: Tensor, target: Tensor) -> None:
#         """Update state with predictions and targets."""
#         sum_squared_error, num_obs = _mean_squared_error_update(preds, target, num_outputs=self.num_outputs)

#         self.sum_squared_error += sum_squared_error
#         self.total += num_obs

#     def compute(self) -> Tensor:
#         """Compute mean squared error over state."""
#         return _mean_squared_error_compute(self.sum_squared_error, self.total, squared=self.squared)

#     def plot(
#         self, val: Optional[Union[Tensor, Sequence[Tensor]]] = None, ax: Optional[_AX_TYPE] = None
#     ) -> _PLOT_OUT_TYPE:
#         """Plot a single or multiple values from the metric.

#         Args:
#             val: Either a single result from calling `metric.forward` or `metric.compute` or a list of these results.
#                 If no value is provided, will automatically call `metric.compute` and plot that result.
#             ax: An matplotlib axis object. If provided will add plot to that axis

#         Returns:
#             Figure and Axes object

#         Raises:
#             ModuleNotFoundError:
#                 If `matplotlib` is not installed

#         .. plot::
#             :scale: 75

#             >>> from torch import randn
#             >>> # Example plotting a single value
#             >>> from torchmetrics.regression import MeanSquaredError
#             >>> metric = MeanSquaredError()
#             >>> metric.update(randn(10,), randn(10,))
#             >>> fig_, ax_ = metric.plot()

#         .. plot::
#             :scale: 75

#             >>> from torch import randn
#             >>> # Example plotting multiple values
#             >>> from torchmetrics.regression import MeanSquaredError
#             >>> metric = MeanSquaredError()
#             >>> values = []
#             >>> for _ in range(10):
#             ...     values.append(metric(randn(10,), randn(10,)))
#             >>> fig, ax = metric.plot(values)

#         """
#         return self._plot(val, ax)

In [3]:
list_y=['THUONG_PHAM_D1M']
list_lag=['SO_LUONG_KH','SO_LUONG_KH_DMTMN','SAN_LUONG_DMTMN','CONG_SUAT_DMTMN']
list_weather=['Average of temp','Min of tempmin','Max of tempmax','Average of dew','Average of precip','Average of humidity','Average of pressure','Average of windspeed','Average of solarradiation','Average of solarenergy']
list_lag2=['NN_1110','NN_1120','NN_1200','NN_1300','NN_1400','NN_2100','NN_2101','NN_2102','NN_2103','NN_2104','NN_2105','NN_2201','NN_2202','NN_2203','NN_2204','NN_2205','NN_2206','NN_2207','NN_2208','NN_2209','NN_2210','NN_2211','NN_2212','NN_2213','NN_2214','NN_2215','NN_2216','NN_2217','NN_2218','NN_2219','NN_2221','NN_2222','NN_2223','NN_2301','NN_2302','NN_2303','NN_2400','NN_2410','NN_2420','NN_2430','NN_2500','NN_3100','NN_3101','NN_3102','NN_3200','NN_3210','NN_3220','NN_3230','NN_4100','NN_4300','NN_4400','NN_4401','NN_4402','NN_5000','NN_5100','NN_5101','NN_5102','NN_5103','NN_5104','NN_5200','NN_5301','NN_5302','NN_5400','NN_5401','NN_5402','NN_5403','NN_5404','NN_5500']
list_lag3=['Holidays','Holidays_date','Holidays_numdays']
list_lag=list_lag+list_lag2+list_lag3+list_weather
list_fu=['NUM_DAY_OF_MONTH','Covid','Ukraine war']
list_static=['LAT','LONG']
# ,'Sum of precip'

In [4]:
df=pd.read_excel('San_luong_thuong_pham_dmtmn_theo_thang_02_05_2024.xlsx')
df['ds'] = pd.PeriodIndex(year=df['NAM'], month=df['THANG'], freq='M')
df['ds'] = df['ds'].dt.to_timestamp()
df.drop(columns=['NAM_THANG','THUONG_PHAM_D1M_TB','THUONG_PHAM','THUONG_PHAM_TB','THUONG_PHAM_T1M','THUONG_PHAM_T1M_TB'],inplace=True)
df.pivot(index='ds',columns='MA_DVIQLY',values='THUONG_PHAM_D1M')

MA_DVIQLY,PC10AA,PC10BB,PC10CC,PC10DD,PC10EE,PC10FF,PC10GG,PC10HH,PC10II,PC10KK,PC10LL,PC10MM,PC10NN,PC10OO,PC10PP
ds,,,,,,,,,,,,,,,
2018-01-01,22780144,5591236,3631087,2147944,6765956,1925927,4200751,5825367,4062065,7787595,1174865,4392775,7386961,2771960,3632473
2018-02-01,20493846,5432752,3411785,2041699,7119443,1845737,3901336,6540228,4279496,8333695,1054739,4251703,7839868,2590108,3543659
2018-03-01,22984151,6000746,4024543,2351662,7674796,2060704,4096243,6461873,4006350,7599260,1228859,4465837,8506157,2983940,3870439
2018-04-01,22867098,6170370,4105717,2531835,6654484,2050945,4076103,4961267,3108473,5564470,1261605,3904932,6895523,3041129,3475606
2018-05-01,23109768,6605146,4258483,2764410,5523007,2189953,3951701,4153447,2827978,4720504,1293530,3358560,5392399,3130344,2951954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,28036226,7040482,5456418,3299510,8774292,2466843,4989979,6922220,4318844,8441099,1809034,5116342,11387319,4351927,4519629
2024-01-01,29890345,7472128,6302058,3390923,10916321,2501838,5511449,9436056,7241823,11305381,1856730,5822724,16669559,4607856,5240909
2024-02-01,26789128,7166829,5681707,3585660,11324567,2683257,5235877,10326069,6460412,11451349,1754988,6804869,15234754,4657306,5228309


In [5]:
dfp=df.pivot(index='ds',columns='MA_DVIQLY',values='THUONG_PHAM_D1M')
dfp['total']=dfp.sum(axis=1)
dfp.tail(4)

MA_DVIQLY,PC10AA,PC10BB,PC10CC,PC10DD,PC10EE,PC10FF,PC10GG,PC10HH,PC10II,PC10KK,PC10LL,PC10MM,PC10NN,PC10OO,PC10PP,total
ds,,,,,,,,,,,,,,,,
2024-01-01,29890345,7472128,6302058,3390923,10916321,2501838,5511449,9436056,7241823,11305381,1856730,5822724,16669559,4607856,5240909,128166100
2024-02-01,26789128,7166829,5681707,3585660,11324567,2683257,5235877,10326069,6460412,11451349,1754988,6804869,15234754,4657306,5228309,124385081
2024-03-01,31633176,8580047,6648549,4537247,12360759,2977369,5910902,10803056,7311590,12887624,2108449,6910177,17774000,5537126,5783013,141763084
2024-04-01,34563110,9287909,7123498,5033673,13290579,3014845,6727819,10769671,7605021,12833665,2287721,6976832,19006172,5908724,6379401,150808640


In [ ]:
# dfp=df.pivot(index='ds',columns='MA_DVIQLY',values='Average of precip')
# dfp['total']=dfp.sum(axis=1)
# dfp.tail(12)

In [ ]:
# df=df.loc[df['ds']<='2024-01-01']
# fu=df[df['ds']>='2023-12-01']
# fu.loc[:,list_y+list_lag]=None
# fu.loc[:,'ds']=fu['ds']+pd.DateOffset(months=+2) #dataframe future 1 tháng
# fu.loc[:,'NAM']=fu['ds'].dt.year
# fu.loc[:,'THANG']=fu['ds'].dt.month
# fu.loc[:,'NUM_DAY_OF_MONTH']=fu['ds'].dt.days_in_month
# df=pd.concat([df,fu])
# df.set_index('ds', inplace=True)
# df.drop(columns='NUM_DAY_OF_MONTH.1',inplace=True)
# df[df['MA_DVIQLY']=='PC10AA']

In [6]:
df=df.loc[df['ds']<='2024-04-01']
fu=df[df['ds']>='2024-03-01']
fu.loc[:,list_y+list_lag]=None
fu.loc[:,'ds']=fu['ds']+pd.DateOffset(months=+2) #dataframe future 1 tháng
fu.loc[:,'NAM']=fu['ds'].dt.year
fu.loc[:,'THANG']=fu['ds'].dt.month
fu.loc[:,'NUM_DAY_OF_MONTH']=fu['ds'].dt.days_in_month
df=pd.concat([df,fu])
df.set_index('ds', inplace=True)
df.drop(columns=['Sum of precip'],inplace=True)
df[df['MA_DVIQLY']=='PC10AA']

,MA_DVIQLY,NAM,THANG,NUM_DAY_OF_MONTH,SO_LUONG_KH,THUONG_PHAM_D1M,SO_LUONG_KH_DMTMN,SAN_LUONG_DMTMN,CONG_SUAT_DMTMN,Average of temp,...,NN_5104,NN_5200,NN_5301,NN_5302,NN_5400,NN_5401,NN_5402,NN_5403,NN_5404,NN_5500
ds,,,,,,,,,,,,,,,,,,,,,
2018-01-01,PC10AA,2018,1,31,75436,22780144,1,175,25,22.13,...,135418,530046,11224,147,0,184835,0,89135,756424,0
2018-02-01,PC10AA,2018,2,28,75778,20493846,0,0,0,21.86,...,123479,496514,10527,100,0,167719,0,93051,698024,0
2018-03-01,PC10AA,2018,3,31,76129,22984151,0,0,0,23.83,...,145696,526196,11401,73,0,214709,0,86394,791259,0
2018-04-01,PC10AA,2018,4,30,76343,22867098,0,0,0,25.62,...,155657,480226,11058,347,0,241104,0,71328,786713,16
2018-05-01,PC10AA,2018,5,31,76290,23109768,0,0,0,25.30,...,172475,475260,11492,410,0,267228,0,76623,818733,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-01,PC10AA,2024,2,29,86897,26789128,888,7164693,64707,23.07,...,396801,774882,0,97235,0,208846,0,8776,725220,670
2024-03-01,PC10AA,2024,3,31,86891,31633176,888,7884863,64707,24.85,...,527451,734422,0,134827,0,281396,0,13195,806856,947
2024-04-01,PC10AA,2024,4,30,87073,34563110,888,7836382,64707,27.96,...,692946,693711,0,125199,0,341160,0,12242,833675,987


In [ ]:
df[df['MA_DVIQLY']=='PC10AA'][list_fu]

In [7]:
y=  TimeSeries.from_group_dataframe(df=df,group_cols='MA_DVIQLY',value_cols=list_y,static_cols=list_static)
lag=TimeSeries.from_group_dataframe(df=df,group_cols='MA_DVIQLY',value_cols=list_lag,static_cols=list_static)
fu=TimeSeries.from_group_dataframe(df=df,group_cols='MA_DVIQLY',value_cols=list_fu,static_cols=list_static)

scaler_y=Scaler()
scaler_lag=Scaler()
scaler_fu=Scaler()
scaler_static_y=StaticCovariatesTransformer()
scaler_static_lag=StaticCovariatesTransformer()
scaler_static_fu=StaticCovariatesTransformer()

y=  scaler_static_y.fit_transform(y)
lag=scaler_static_lag.fit_transform(lag)
fu= scaler_static_fu.fit_transform(fu)

nom_y=  scaler_y.fit_transform(y)
nom_lag=scaler_lag.fit_transform(lag)
nom_fu= scaler_fu.fit_transform(fu)


In [ ]:
# test=[8,7,6,5,4,3,2,1]
# test[-6:],test[:-1],test[-6:-1]

In [ ]:
# back_step=1
# n_step=2
# len_val=35

# train_y  =[i[:-n_step-back_step] for i in nom_y]
# train_lag=[i[:-n_step-back_step] for i in nom_lag]
# train_fu =[i[:-back_step] for i in nom_fu] if back_step>0 else [i for i in nom_fu]

# val_y    =[i[-len_val-back_step:-n_step-back_step] for i in nom_y]
# val_lag  =[i[-len_val-back_step:-n_step-back_step] for i in nom_lag]
# val_fu   =[i[-len_val-back_step:-back_step] for i in nom_fu] if back_step>0 else [i[-len_val:] for i in nom_fu]

# val_metric=[i[-n_step-back_step:-back_step] for i in nom_y] if back_step>0 else [i[-n_step:] for i in nom_y]

In [ ]:
nom_fu[0].pd_dataframe()

In [ ]:
nom_lag[1].static_covariates

In [ ]:
lag[4].static_covariates

In [ ]:
# %%time
# model = TiDEModel(
#     input_chunk_length=25,
#     output_chunk_length=2,
#     n_epochs=50,
#     random_state=0,
#     batch_size=64,
#     model_name='my_model', save_checkpoints=True,
#     force_reset=True,
#     use_reversible_instance_norm=True,
#     **generate_torch_kwargs()
# )

# # model.trainer_params["enable_progress_bar"] = False
# model.fit(train_y,train_lag,train_fu,val_y,val_lag,val_fu)
# print(glob.glob('/kaggle/working/*/*/*/*'))
# # model.fit(nom_y,past_covariates=nom_lag,future_covariates=nom_fu)
# best_model = model.load_from_checkpoint(model_name='my_model', best=True)
# pred_list = scaler_y.inverse_transform(best_model.predict(n=2, series=train_y, past_covariates=train_lag, future_covariates=train_fu))
# total=concatenate(pred_list,axis=1).pd_dataframe()
# total['PC10']=total.sum(axis=1)
# total

In [ ]:
# dfp.tail(6)

In [ ]:
%pdb on

In [ ]:
# print(glob.glob('/kaggle/working/*/*/*/*'))

In [ ]:
# results=None

# for i in range(0,12):
#     back_step=i
#     n_step=2
#     len_val=35

#     train_y  =[i[:-n_step-back_step] for i in nom_y]
#     train_lag=[i[:-n_step-back_step] for i in nom_lag]
#     train_fu =[i[:-back_step] for i in nom_fu] if back_step>0 else [i for i in nom_fu]

#     val_y    =[i[-len_val-back_step:-n_step-back_step] for i in nom_y]
#     val_lag  =[i[-len_val-back_step:-n_step-back_step] for i in nom_lag]
#     val_fu   =[i[-len_val-back_step:-back_step] for i in nom_fu] if back_step>0 else [i[-len_val:] for i in nom_fu]

#     val_metric=[i[-n_step-back_step:-back_step] for i in nom_y] if back_step>0 else [i[-n_step:] for i in nom_y]

#     model = TiDEModel(
#         input_chunk_length=25,
#         output_chunk_length=2,
#         n_epochs=20,
#         num_encoder_layers=3,
#         num_decoder_layers=3,
#         decoder_output_dim=32,
#         random_state=10,
#         batch_size=4,
#         dropout=0.2,
#         optimizer_kwargs={"lr": 5e-5},
#         model_name='my_model', save_checkpoints=True,
#         force_reset=True,
#         use_reversible_instance_norm=True,
#         **generate_torch_kwargs()
#     )

#     # model.trainer_params["enable_progress_bar"] = False
#     model.fit(train_y,train_lag,train_fu,val_y,val_lag,val_fu)
#     print(glob.glob('/kaggle/working/*/*/*/*'))
# #     model.fit(nom_y,past_covariates=nom_lag,future_covariates=nom_fu)
#     best_model = model.load_from_checkpoint(model_name='my_model', best=True)
#     pred_list = scaler_y.inverse_transform(best_model.predict(n=2, series=train_y, past_covariates=train_lag, future_covariates=train_fu))
#     total=concatenate(pred_list,axis=1).pd_dataframe()
#     total['PC10']=total.sum(axis=1)
#     total=total.tail(1)
    
#     results = total if results is None else pd.concat([total,results])
#     print(total['PC10'])

In [ ]:
# from lightning.pytorch.callbacks import Callback

# class MyCallback(Callback):
#     def on_validation_end(self, trainer, pl_module):
#         print('Validation epoch {} ended'.format(trainer.current_epoch))

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import Callback
from lightning.pytorch.demos.boring_classes import BoringDataModule, BoringModel

In [ ]:
# class PredictAfterValidationCallback(Callback):
#     def setup(self, trainer, pl_module, stage):
#         if stage in ("fit", "validate"):
#             # setup the predict data even for fit/validate, as we will call it during `on_validation_epoch_end`
#             trainer.datamodule.setup("validate")

#     def on_validation_epoch_end(self, trainer, pl_module):
#         if trainer.sanity_checking:  # optional skip
#             return
#         print("Start predicting!")
#         # this will fetch the dataloader from the LM or LDM
#         predict_dataloader = trainer._data_connector._predict_dataloader_source.dataloader()
#         for i, batch in enumerate(predict_dataloader):
#             # this will move the batch to device using the LM or LDM
#             batch = pl_module._apply_batch_transfer_handler(batch)
#             # manually run the predict step
#             out = pl_module.predict_step(batch, i)
#             print(i, out)

In [ ]:
from darts.utils.losses import MapeLoss

In [ ]:
from darts.metrics.metrics import rmse

In [13]:
results1=None
results2=None
# metric=torchmetrics.MeanAbsoluteError()
metric=torchmetrics.MeanAbsoluteError()
for i in range(0,8):
    back_step=i
    n_step=2
    len_val=4
    input_chunk_length=6

    train_y  =[i[:-n_step-back_step-len_val+len_val] for i in nom_y]
    train_lag=[i[:-n_step-back_step-len_val+len_val] for i in nom_lag]
    train_fu =[i[:-back_step-len_val+len_val] for i in nom_fu] if back_step>0 else [i for i in nom_fu]

    val_y    =[i[-len_val-input_chunk_length-back_step:-n_step-back_step] for i in nom_y]
    val_lag  =[i[-len_val-input_chunk_length-back_step:-n_step-back_step] for i in nom_lag]
    val_fu   =[i[-len_val-input_chunk_length-back_step:-back_step] for i in nom_fu] \
                if back_step>0 else [i[-len_val-input_chunk_length:] for i in nom_fu]

    test_y  =[i[:-n_step-back_step] for i in nom_y]
    test_lag=[i[:-n_step-back_step] for i in nom_lag]
    test_fu =[i[:-back_step] for i in nom_fu] if back_step>0 else [i for i in nom_fu]

    val_metric=[i[-n_step-back_step:-back_step] for i in nom_y] if back_step>0 else [i[-n_step:] for i in nom_y]
 
    model = iTransformerModel(
        input_chunk_length=input_chunk_length,
        output_chunk_length=n_step,
        n_epochs=15,
        num_encoder_layers=3,
        num_decoder_layers=3,
        d_model=64,
        # decoder_output_dim=32,
        torch_metrics=metric,
        # hidden_size=128,
#         use_layer_norm=True,
        loss_fn=CustomLoss(),
        random_state=100,
        batch_size=16,
        dropout=0.2,
        activation='SwiGLU',
        monitor_checkpoint="val_MeanAbsoluteError",
#         use_layer_norm=True,
        optimizer_cls=torch.optim.AdamW,
        optimizer_kwargs={"lr": 1e-3,
                          "amsgrad":True,
                          "eps":1e-12},
        model_name='my_model', save_checkpoints=True,
        force_reset=True,
        use_reversible_instance_norm=True,
        pl_trainer_kwargs= {
            "accelerator": "cpu",
            "callbacks": [TFMProgressBar(enable_train_bar_only=True,refresh_rate=50,)],
            "limit_train_batches":0.2,
        },
    )

    # model.trainer_params["enable_progress_bar"] = False
 
    model.fit(train_y,train_lag,None,val_y,val_lag,None)
    print(sorted(glob.glob('/kaggle/working/*/*/*/*')))
    best_model = model.load_from_checkpoint(model_name='my_model', best=True)
    pred_list = scaler_static_y.inverse_transform(scaler_y.inverse_transform(best_model.predict(n=n_step, series=test_y, past_covariates=test_lag, future_covariates=None)))
#     total=concatenate(pred_list,axis=0).pd_dataframe()
    total=pd.concat([pred_list[i]['THUONG_PHAM_D1M'].pd_dataframe() for i in range(len(pred_list))],axis=1,)
    total.columns=[pred_list[i].static_covariates['MA_DVIQLY']['THUONG_PHAM_D1M'] for i in range(len(pred_list))]
    total['PC10']=total.sum(axis=1)
    total=total.tail(2)
    
    results1 = total.head(1) if results1 is None else pd.concat([total.head(1),results1])
    results2 = total.tail(1) if results2 is None else pd.concat([total.tail(1),results2])
    print(total['PC10'])

Epoch 14: 100%|██████████| 13/13 [00:01<00:00,  9.27it/s, train_loss=0.0102, val_loss=0.0401, val_MeanAbsoluteError=0.163, train_MeanAbsoluteError=0.096]
[]
ds
2024-05-01    1.445579e+08
2024-06-01    1.413813e+08
Freq: MS, Name: PC10, dtype: float64
Epoch 14: 100%|██████████| 12/12 [00:01<00:00,  9.37it/s, train_loss=0.00818, val_loss=0.0192, val_MeanAbsoluteError=0.113, train_MeanAbsoluteError=0.105]
[]
ds
2024-04-01    1.473120e+08
2024-05-01    1.470121e+08
Freq: MS, Name: PC10, dtype: float64
Epoch 6:   0%|          | 0/12 [00:00<?, ?it/s, train_loss=0.0316, val_loss=0.0447, val_MeanAbsoluteError=0.139, train_MeanAbsoluteError=0.104]         []
ds
2024-03-01    1.373352e+08
2024-04-01    1.378257e+08
Freq: MS, Name: PC10, dtype: float64
Epoch 7:   0%|          | 0/12 [00:00<?, ?it/s, train_loss=0.0104, val_loss=0.113, val_MeanAbsoluteError=0.210, train_MeanAbsoluteError=0.121]         

In [ ]:
import glob

In [ ]:
glob.glob("./darts_logs/my_model/checkpoints/*")

In [ ]:
# results1=None
# results2=None

# for i in range(0,18):
#     back_step=i
#     n_step=2
#     len_val=4
#     input_chunk_length=25

#     train_y  =[i[:-n_step-back_step-len_val+len_val] for i in nom_y]
#     train_lag=[i[:-n_step-back_step-len_val+len_val] for i in nom_lag]
#     train_fu =[i[:-back_step-len_val+len_val] for i in nom_fu] if back_step>0 else [i for i in nom_fu]

#     val_y    =[i[-len_val-input_chunk_length-back_step:-n_step-back_step] for i in nom_y]
#     val_lag  =[i[-len_val-input_chunk_length-back_step:-n_step-back_step] for i in nom_lag]
#     val_fu   =[i[-len_val-input_chunk_length-back_step:-back_step] for i in nom_fu] \
#                 if back_step>0 else [i[-len_val-input_chunk_length:] for i in nom_fu]
#     test_y  =[i[:-n_step-back_step] for i in nom_y]
#     test_lag=[i[:-n_step-back_step] for i in nom_lag]
#     test_fu =[i[:-back_step] for i in nom_fu] if back_step>0 else [i for i in nom_fu]

#     val_metric=[i[-n_step-back_step:-back_step] for i in nom_y] if back_step>0 else [i[-n_step:] for i in nom_y]
# #     ipdb.set_trace()
#     model = TFTModel(
#         input_chunk_length=input_chunk_length,
#         output_chunk_length=n_step,
#         n_epochs=200,
# #         num_encoder_layers=5,
# #         num_decoder_layers=5,
# #         decoder_output_dim=32,
#         torch_metrics=torchmetrics.MeanAbsolutePercentageError(),
# #         hidden_size=32,
# #         use_layer_norm=True,
# #         loss_fn=LogCoshLoss(),
#         random_state=50,
#         batch_size=32,
#         dropout=0.2,
# #         activation="SwiGLU",
# #         use_layer_norm=True,
#         optimizer_cls=torch.optim.AdamW,
#         optimizer_kwargs={"lr": 5e-5,
#                           "amsgrad":True,
#                           "eps":1e-10},
#         model_name='my_model', save_checkpoints=True,
#         force_reset=True,
#         use_reversible_instance_norm=True,
#         pl_trainer_kwargs= {
#             "accelerator": "cpu",
#             "callbacks": [TFMProgressBar(enable_train_bar_only=True,refresh_rate=50,)],
# #             "limit_train_batches":0.2,
#         },
#     )

#     # model.trainer_params["enable_progress_bar"] = False
#     model.fit(train_y,past_covariates=train_lag,future_covariates=train_fu,val_series=val_y,val_past_covariates=val_lag,val_future_covariates=val_fu,)
#     print(sorted(glob.glob('/kaggle/working/*/*/*/*')))
#     best_model = model.load_from_checkpoint(model_name='my_model', best=True)
#     pred_list = scaler_static_y.inverse_transform(scaler_y.inverse_transform(model.predict(n=n_step, series=test_y, past_covariates=test_lag)))
# #     total=concatenate(pred_list,axis=0).pd_dataframe()
#     total=pd.concat([pred_list[i]['THUONG_PHAM_D1M'].pd_dataframe() for i in range(len(pred_list))],axis=1,)
#     total.columns=[pred_list[i].static_covariates['MA_DVIQLY']['THUONG_PHAM_D1M'] for i in range(len(pred_list))]
#     total['PC10']=total.sum(axis=1)
#     total=total.tail(2)
    
#     results1 = total.head(1) if results1 is None else pd.concat([total.head(1),results1])
#     results2 = total.tail(1) if results2 is None else pd.concat([total.tail(1),results2])
#     print(total['PC10'])

In [ ]:
pred_list = scaler_static_y.inverse_transform(scaler_y.inverse_transform(model.predict(n=n_step, series=test_y, past_covariates=test_lag, future_covariates=test_fu)))

In [ ]:
total=pd.concat([pred_list[i]['THUONG_PHAM_D1M'].pd_dataframe() for i in range(len(pred_list))],axis=1,)
total.columns=[pred_list[i].static_covariates['MA_DVIQLY']['THUONG_PHAM_D1M'] for i in range(len(pred_list))]
total['PC10']=total.sum(axis=1)
total

In [ ]:
pred_list = scaler_y.inverse_transform(model.predict(n=n_step, series=test_y, past_covariates=test_lag, future_covariates=test_fu))
total=concatenate(pred_list,axis=1).pd_dataframe()
total['PC10']=total.sum(axis=1)
total=total.tail(2)
total

In [ ]:
A=pd.concat([dfp.tail(19)['total'],results1['PC10']],axis=1)
A['sai_so_1T_%']=(A['PC10']-A['total'])/A['PC10']*100
A=A.rename(columns={'total':'thuc_te','PC10':'du_bao_truoc_1T'})
A=pd.concat([A,results2['PC10']],axis=1)
A['sai_so_2T_%']=(A['PC10']-A['thuc_te'])/A['PC10']*100
A=A.rename(columns={'PC10':'du_bao_truoc_2T'})
A

In [ ]:
# model = TiDEModel(
#     input_chunk_length=15,
#     output_chunk_length=2,
#     n_epochs=6,
# #     random_state=20,
#     batch_size=1,
#     dropout=0.1,
#     optimizer_kwargs={"lr": 1e-4},
#     model_name='my_model', save_checkpoints=True,
#     force_reset=True,
#     use_reversible_instance_norm=True,
#     **generate_torch_kwargs()
# )

# # model.trainer_params["enable_progress_bar"] = False
# model.fit(train_y,train_lag,train_fu,val_y,val_lag,val_fu)
# print(glob.glob('/kaggle/working/*/*/*/*'))
# # model.fit(nom_y,past_covariates=nom_lag,future_covariates=nom_fu)
# best_model = model.load_from_checkpoint(model_name='my_model', best=True)
# pred_list = scaler_y.inverse_transform(best_model.predict(n=2, series=train_y, past_covariates=train_lag, future_covariates=train_fu))
# total=concatenate(pred_list,axis=1).pd_dataframe()
# total['PC10']=total.sum(axis=1)
# total

In [ ]:
# best_model = model.load_from_checkpoint(model_name='my_model', best=False)
# pred_list = scaler_y.inverse_transform(best_model.predict(n=2, series=train_y, past_covariates=train_lag, future_covariates=train_fu))
# total=concatenate(pred_list,axis=1).pd_dataframe()
# total['PC10']=total.sum(axis=1)
# total


In [ ]:
from lightning.pytorch import LightningModule
from lightning.pytorch import Trainer
from lightning.pytorch import Callback
import optuna

class PyTorchLightningPruningCallback(Callback):
    """PyTorch Lightning callback to prune unpromising trials.
    See `the example <https://github.com/optuna/optuna-examples/blob/
    main/pytorch/pytorch_lightning_simple.py>`__
    if you want to add a pruning callback which observes accuracy.
    Args:
        trial:
            A :class:`~optuna.trial.Trial` corresponding to the current evaluation of the
            objective function.
        monitor:
            An evaluation metric for pruning, e.g., ``val_loss`` or
            ``val_acc``. The metrics are obtained from the returned dictionaries from e.g.
            ``pytorch_lightning.LightningModule.training_step`` or
            ``pytorch_lightning.LightningModule.validation_epoch_end`` and the names thus depend on
            how this dictionary is formatted.
    """

    def __init__(self, trial: optuna.trial.Trial, monitor: str) -> None:
        super().__init__()

        self._trial = trial
        self.monitor = monitor
        self._best_score = None

    def on_validation_end(self, trainer: Trainer, pl_module: LightningModule) -> None:
        # When the trainer calls `on_validation_end` for sanity check,
        # do not call `trial.report` to avoid calling `trial.report` multiple times
        # at epoch 0. The related page is
        # https://github.com/PyTorchLightning/pytorch-lightning/issues/1391.
        if trainer.sanity_checking:
            return

        epoch = pl_module.current_epoch

        current_score = trainer.callback_metrics.get(self.monitor)
        if current_score is None:
            message = (
                "The metric '{}' is not in the evaluation logs for pruning. "
                "Please make sure you set the correct metric name.".format(self.monitor)
            )
            warnings.warn(message)
            return
        if self._best_score is None or current_score < self._best_score:
            self._best_score = current_score
            trainer.save_checkpoint("best_checkpoint.ckpt")
            
        self._trial.report(current_score, step=epoch)
        if self._trial.should_prune() and epoch>0:
            message = "Trial was pruned at epoch {}.".format(epoch)
            raise optuna.TrialPruned(message)
    def on_train_end(self, trainer, pl_module):
        print(f"epoch {pl_module.current_epoch}",end=" ")

In [ ]:
import numpy as np
from lightning.pytorch.callbacks import EarlyStopping
import optuna
# from optuna.integration import PyTorchLightningPruningCallback
from darts.metrics import smape

back_step=2
n_step=2
len_val=4
input_chunk_length=25

train_y  =[i[:-n_step-back_step-len_val+len_val] for i in nom_y]
train_lag=[i[:-n_step-back_step-len_val+len_val] for i in nom_lag]
train_fu =[i[:-back_step-len_val+len_val] for i in nom_fu] if back_step>0 else [i for i in nom_fu]

val_y    =[i[-len_val-input_chunk_length-back_step:-n_step-back_step] for i in nom_y]
val_lag  =[i[-len_val-input_chunk_length-back_step:-n_step-back_step] for i in nom_lag]
val_fu   =[i[-len_val-input_chunk_length-back_step:-back_step] for i in nom_fu] \
            if back_step>0 else [i[-len_val-input_chunk_length:] for i in nom_fu]

test_y  =[i[:-n_step-back_step] for i in nom_y]
test_lag=[i[:-n_step-back_step] for i in nom_lag]
test_fu =[i[:-back_step] for i in nom_fu] if back_step>0 else [i for i in nom_fu]

val_metric=[i[-n_step-back_step:-back_step] for i in nom_y] \
            if back_step>0 else [i[-n_step:] for i in nom_y]

# define objective function
def objective(trial):
    # select input and output chunk lengths
    n_epochs = trial.suggest_int("n_epochs", 10, 40)

    # Other hyperparameters
#     dropout = trial.suggest_float("dropout", 0.1, 0.2)
    lr = trial.suggest_float("lr", 5e-5, 1e-3, log=True)

    # throughout training we'll monitor the validation loss for both pruning and early stopping
    pruner = PyTorchLightningPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.0001, patience=8, verbose=True)

    # reproducibility
    torch.manual_seed(42)

    # build the TCN model
    model = TiDEModel(
        input_chunk_length=input_chunk_length,
        output_chunk_length=n_step,
        n_epochs=36,
        num_encoder_layers=5,
        num_decoder_layers=5,
        decoder_output_dim=32,
        random_state=20,
        batch_size=4,
        dropout=0.2,
        activation=SwiGLU,
        optimizer_kwargs={"lr": lr},
        model_name='my_model', save_checkpoints=True,
        force_reset=True,
        use_reversible_instance_norm=True,
        pl_trainer_kwargs= {
            "accelerator": "cpu",
            "callbacks": [TFMProgressBar(enable_train_bar_only=True,refresh_rate=50,)],
            "limit_train_batches":0.2,
        }
    )


    # when validating during training, we can use a slightly longer validation
    # set which also contains the first input_chunk_length time steps
    # train the model
    
    model.fit(train_y,train_lag,train_fu,val_y,val_lag,val_fu,verbose=True)

    # reload best model over course of training
#     model = TiDEModel.load_from_checkpoint(model_name='my_model')

    # Evaluate how good it is on the validation set, using sMAPE
    preds = model.predict(n=2, series=train_y, past_covariates=train_lag, future_covariates=train_fu)
    smapes = smape(val_metric, preds, n_jobs=-1)
    smape_val = np.mean(smapes)
    
#     ipdb.set_trace()
    
    return smape_val if smape_val != np.nan else float("inf")

# for convenience, print some optimization trials information
def print_callback(study, trial):
    trial_params = {k: format(v, '.4e') for k, v in trial.params.items()}
    study_best_trial_params={k: format(v, '.4e') for k, v in study.best_trial.params.items()}
    print(f"Current value: {trial.value:.5e},params: {trial_params}")
    print(f"Best value: {study.best_value:.5e},params: {study_best_trial_params}")

# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=15, callbacks=[print_callback])

In [ ]:
# import numpy as np
# import torch
# from pytorch_lightning.callbacks import EarlyStopping
# from sklearn.preprocessing import MaxAbsScaler

# from darts.dataprocessing.transformers import Scaler
# from darts.datasets import AirPassengersDataset
# from darts.metrics import smape
# from darts.models import TCNModel
# from darts.utils.likelihood_models import GaussianLikelihood

# # load data
# series = AirPassengersDataset().load().astype(np.float32)

# # split in train / validation (note: in practice we would also need a test set)
# VAL_LEN = 36
# train, val = series[:-VAL_LEN], series[-VAL_LEN:]

# # scale
# scaler = Scaler(MaxAbsScaler())
# train = scaler.fit_transform(train)
# val = scaler.transform(val)

# # define objective function
# def objective(trial):
#     # select input and output chunk lengths
#     in_len = trial.suggest_int("in_len", 12, 36)
#     out_len = trial.suggest_int("out_len", 1, in_len-1)

#     # Other hyperparameters
#     kernel_size = trial.suggest_int("kernel_size", 2, 5)
#     num_filters = trial.suggest_int("num_filters", 1, 5)
#     weight_norm = trial.suggest_categorical("weight_norm", [False, True])
#     dilation_base = trial.suggest_int("dilation_base", 2, 4)
#     dropout = trial.suggest_float("dropout", 0.0, 0.4)
#     lr = trial.suggest_float("lr", 5e-5, 1e-3, log=True)
#     include_year = trial.suggest_categorical("year", [False, True])

#     # throughout training we'll monitor the validation loss for both pruning and early stopping
#     pruner = PyTorchLightningPruningCallback(trial, monitor="val_loss")
#     early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=3, verbose=True)
#     callbacks = [pruner, early_stopper]

#     # detect if a GPU is available
#     if torch.cuda.is_available():
#         num_workers = 4
#     else:
#         num_workers = 0

#     pl_trainer_kwargs = {
#         "accelerator": "auto",
#         "callbacks": callbacks,
#     }

#     # optionally also add the (scaled) year value as a past covariate
#     if include_year:
#         encoders = {"datetime_attribute": {"past": ["year"]},
#                     "transformer": Scaler()}
#     else:
#         encoders = None

#     # reproducibility
#     torch.manual_seed(42)

#     # build the TCN model
#     model = TCNModel(
#         input_chunk_length=in_len,
#         output_chunk_length=out_len,
#         batch_size=32,
#         n_epochs=100,
#         nr_epochs_val_period=1,
#         kernel_size=kernel_size,
#         num_filters=num_filters,
#         weight_norm=weight_norm,
#         dilation_base=dilation_base,
#         dropout=dropout,
#         optimizer_kwargs={"lr": lr},
#         add_encoders=encoders,
#         likelihood=GaussianLikelihood(),
#         pl_trainer_kwargs=pl_trainer_kwargs,
#         model_name="tcn_model",
#         force_reset=True,
#         save_checkpoints=True,
#     )


#     # when validating during training, we can use a slightly longer validation
#     # set which also contains the first input_chunk_length time steps
#     model_val_set = scaler.transform(series[-(VAL_LEN + in_len) :])

#     # train the model
#     model.fit(
#         series=train,
#         val_series=model_val_set,
#         num_loader_workers=num_workers,
#     )

#     # reload best model over course of training
#     model = TCNModel.load_from_checkpoint("tcn_model")

#     # Evaluate how good it is on the validation set, using sMAPE
#     preds = model.predict(series=train, n=VAL_LEN)
#     smapes = smape(val, preds, n_jobs=-1, verbose=True)
#     smape_val = np.mean(smapes)
    
#     return smape_val if smape_val != np.nan else float("inf")


# # for convenience, print some optimization trials information
# def print_callback(study, trial):
#     print(f"Current value: {trial.value}, Current params: {trial.params}")
#     print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")


# # optimize hyperparameters by minimizing the sMAPE on the validation set
# if __name__ == "__main__":
#     study = optuna.create_study(direction="minimize")
#     study.optimize(objective, n_trials=100, callbacks=[print_callback])

In [ ]:
# pred_list = model_air_milk.predict(n=36, series=[train_air, train_milk])
# for series, label in zip(pred_list, ["air passengers", "milk production"]):
#     series.plot(label=f"forecast {label}")
# plt.legend()

In [ ]:
# # generate an DataFrame example
# df = pd.DataFrame(
#     data={
#         "dates": [
#             "2020-01-01",
#             "2020-01-02",
#             "2020-01-03",
#             "2020-01-01",
#             "2020-01-02",
#             "2020-01-03",
#         ],
#         "comp1": np.random.random((6,)),
#         "comp2": np.random.random((6,)),
#         "comp3": np.random.random((6,)),
#         "ID": ["SERIES1", "SERIES1", "SERIES1", "SERIES2", "SERIES2", "SERIES2"],
#         "var1": [0.5, 0.5, 0.5, 0.75, 0.75, 0.75],
#     }
# )
# df

In [ ]:
# series_multi = TimeSeries.from_group_dataframe(
#     df,
#     time_col="dates",
#     group_cols="ID",  # individual time series are extracted by grouping `df` by `group_cols`
#     static_cols=[
#         "var1"
#     ],  # also extract these additional columns as static covariates (without grouping)
#     value_cols=[
#         "comp1",
#         "comp2",
#         "comp3",
#     ],  # optionally, specify the time varying columns
# )

# print(f"\n{len(series_multi)} series were extracted from the input DataFrame")
# for i, ts in enumerate(series_multi):
#     print(f"Static covariates of series {i}")
#     print(ts.static_covariates)
# #     ts["comp1"].plot(label=f"comp1_series_{i}")

In [ ]:
# from darts.dataprocessing.transformers import StaticCovariatesTransformer

# transformer = StaticCovariatesTransformer()
# series_transformed = transformer.fit_transform(series_multi)

# for i, (ts, ts_scaled) in enumerate(zip(series_multi, series_transformed)):
#     print(f"Original series {i}")
#     print(ts.static_covariates)
#     print(f"Transformed series {i}")
#     print(ts_scaled.static_covariates)
#     print("")

In [ ]:
# fu=df[df['ds']>='2024-01-01']
# fu.loc[:,'y']=None
# fu.loc[:,'DAU_NGUON_NHAN']=None
# fu.loc[:,'ds']=fu['ds']+pd.DateOffset(months=+2)
# # fu.loc[:,'NAM']=fu['ds'].dt.year
# # fu.loc[:,'THANG']=fu['ds'].dt.month
# fu.loc[:,'NUM_DAY_OF_MONTH']=fu['ds'].dt.days_in_month
# fu
# fu=pd.concat([df,fu])
# fu

In [ ]:
# from datetime import date 
# import holidays 
# import pandas as pd

# # Select country   
# # Print all the holidays in Vietnam in year 2022 
# holidays_list = [(date, name) for date, name in holidays.Vietnam(years = [2018,2019,2020,2021,2022,2023,2024]).items()]

# # Convert list to DataFrame
# df = pd.DataFrame(holidays_list, columns=['Date', 'Holiday'])

# # Print DataFrame
# df.to_csv('ngay_le.csv')

In [ ]:
# """
# Optuna example that optimizes multi-layer perceptrons using PyTorch Lightning.

# In this example, we optimize the validation accuracy of fashion product recognition using
# PyTorch Lightning, and FashionMNIST. We optimize the neural network architecture. As it is too time
# consuming to use the whole FashionMNIST dataset, we here use a small subset of it.

# You can run this example as follows, pruning can be turned on and off with the `--pruning`
# argument.
#     $ python pytorch_lightning_simple.py [--pruning]

# """

# import argparse
# import os
# from typing import List
# from typing import Optional

# import lightning.pytorch as pl
# import optuna
# from optuna.integration import PyTorchLightningPruningCallback
# from packaging import version
# import torch
# from torch import nn
# from torch import optim
# import torch.nn.functional as F
# from torch.utils.data import DataLoader
# from torch.utils.data import random_split
# from torchvision import datasets
# from torchvision import transforms


# if version.parse(pl.__version__) < version.parse("1.6.0"):
#     raise RuntimeError("PyTorch Lightning>=1.6.0 is required for this example.")

# PERCENT_VALID_EXAMPLES = 0.1
# BATCHSIZE = 128
# CLASSES = 10
# EPOCHS = 10
# DIR = os.getcwd()


# class Net(nn.Module):
#     def __init__(self, dropout: float, output_dims: List[int]) -> None:
#         super().__init__()
#         layers: List[nn.Module] = []

#         input_dim: int = 28 * 28
#         for output_dim in output_dims:
#             layers.append(nn.Linear(input_dim, output_dim))
#             layers.append(nn.ReLU())
#             layers.append(nn.Dropout(dropout))
#             input_dim = output_dim

#         layers.append(nn.Linear(input_dim, CLASSES))

#         self.layers = nn.Sequential(*layers)

#     def forward(self, data: torch.Tensor) -> torch.Tensor:
#         logits = self.layers(data)
#         return F.log_softmax(logits, dim=1)


# class LightningNet(pl.LightningModule):
#     def __init__(self, dropout: float, output_dims: List[int]) -> None:
#         super().__init__()
#         self.model = Net(dropout, output_dims)

#     def forward(self, data: torch.Tensor) -> torch.Tensor:
#         return self.model(data.view(-1, 28 * 28))

#     def training_step(self, batch: List[torch.Tensor], batch_idx: int) -> torch.Tensor:
#         data, target = batch
#         output = self(data)
#         return F.nll_loss(output, target)

#     def validation_step(self, batch: List[torch.Tensor], batch_idx: int) -> None:
#         data, target = batch
#         output = self(data)
#         pred = output.argmax(dim=1, keepdim=True)
#         accuracy = pred.eq(target.view_as(pred)).float().mean()
#         self.log("val_acc", accuracy)
#         self.log("hp_metric", accuracy, on_step=False, on_epoch=True)

#     def configure_optimizers(self) -> optim.Optimizer:
#         return optim.Adam(self.model.parameters())


# class FashionMNISTDataModule(pl.LightningDataModule):
#     def __init__(self, data_dir: str, batch_size: int):
#         super().__init__()
#         self.data_dir = data_dir
#         self.batch_size = batch_size

#     def setup(self, stage: Optional[str] = None) -> None:
#         self.mnist_test = datasets.FashionMNIST(
#             self.data_dir, train=False, download=True, transform=transforms.ToTensor()
#         )
#         mnist_full = datasets.FashionMNIST(
#             self.data_dir, train=True, download=True, transform=transforms.ToTensor()
#         )
#         self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

#     def train_dataloader(self) -> DataLoader:
#         return DataLoader(
#             self.mnist_train, batch_size=self.batch_size, shuffle=True, pin_memory=True
#         )

#     def val_dataloader(self) -> DataLoader:
#         return DataLoader(
#             self.mnist_val, batch_size=self.batch_size, shuffle=False, pin_memory=True
#         )

#     def test_dataloader(self) -> DataLoader:
#         return DataLoader(
#             self.mnist_test, batch_size=self.batch_size, shuffle=False, pin_memory=True
#         )


# def objective(trial: optuna.trial.Trial) -> float:
#     # We optimize the number of layers, hidden units in each layer and dropouts.
#     n_layers = trial.suggest_int("n_layers", 1, 3)
#     dropout = trial.suggest_float("dropout", 0.2, 0.5)
#     output_dims = [
#         trial.suggest_int("n_units_l{}".format(i), 4, 128, log=True) for i in range(n_layers)
#     ]

#     model = LightningNet(dropout, output_dims)
#     datamodule = FashionMNISTDataModule(data_dir=DIR, batch_size=BATCHSIZE)

#     trainer = pl.Trainer(
#         logger=True,
#         limit_val_batches=PERCENT_VALID_EXAMPLES,
#         enable_checkpointing=False,
#         max_epochs=EPOCHS,
#         accelerator="auto",
#         devices=1,
#         callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_acc")],
#     )
#     hyperparameters = dict(n_layers=n_layers, dropout=dropout, output_dims=output_dims)
#     trainer.logger.log_hyperparams(hyperparameters)
#     trainer.fit(model, datamodule=datamodule)

#     return trainer.callback_metrics["val_acc"].item()


# if __name__ == "__main__":

#     pruner = optuna.pruners.MedianPruner() 

#     study = optuna.create_study(direction="maximize", pruner=pruner)
#     study.optimize(objective, n_trials=100, timeout=600)

#     print("Number of finished trials: {}".format(len(study.trials)))

#     print("Best trial:")
#     trial = study.best_trial

#     print("  Value: {}".format(trial.value))

#     print("  Params: ")
#     for key, value in trial.params.items():
#         print("    {}: {}".format(key, value))

In [ ]:
# from pytorch_lightning import LightningModule
# from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks import Callback
# import optuna

# class PyTorchLightningPruningCallback(Callback):
#     """PyTorch Lightning callback to prune unpromising trials.
#     See `the example <https://github.com/optuna/optuna-examples/blob/
#     main/pytorch/pytorch_lightning_simple.py>`__
#     if you want to add a pruning callback which observes accuracy.
#     Args:
#         trial:
#             A :class:`~optuna.trial.Trial` corresponding to the current evaluation of the
#             objective function.
#         monitor:
#             An evaluation metric for pruning, e.g., ``val_loss`` or
#             ``val_acc``. The metrics are obtained from the returned dictionaries from e.g.
#             ``pytorch_lightning.LightningModule.training_step`` or
#             ``pytorch_lightning.LightningModule.validation_epoch_end`` and the names thus depend on
#             how this dictionary is formatted.
#     """

#     def __init__(self, trial: optuna.trial.Trial, monitor: str) -> None:
#         super().__init__()

#         self._trial = trial
#         self.monitor = monitor

#     def on_validation_end(self, trainer: Trainer, pl_module: LightningModule) -> None:
#         # When the trainer calls `on_validation_end` for sanity check,
#         # do not call `trial.report` to avoid calling `trial.report` multiple times
#         # at epoch 0. The related page is
#         # https://github.com/PyTorchLightning/pytorch-lightning/issues/1391.
#         if trainer.sanity_checking:
#             return

#         epoch = pl_module.current_epoch

#         current_score = trainer.callback_metrics.get(self.monitor)
#         if current_score is None:
#             message = (
#                 "The metric '{}' is not in the evaluation logs for pruning. "
#                 "Please make sure you set the correct metric name.".format(self.monitor)
#             )
#             warnings.warn(message)
#             return

#         self._trial.report(current_score, step=epoch)
#         if self._trial.should_prune():
#             message = "Trial was pruned at epoch {}.".format(epoch)
#             raise optuna.TrialPruned(message)
#     def on_train_end(self, trainer, pl_module):
#         print(pl_module.current_epoch)